In [1]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
import numpy as np
import math
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from PIL import Image
import random
import keras
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.models import Sequential
import keras.backend as K

Using TensorFlow backend.


In [2]:
def file2array(path, delimiter):
    recordlist = []
    fp = open(path, 'r', encoding='utf-8')
    content = fp.read()     # content现在是一行字符串，该字符串包含文件所有内容
    fp.close()
    rowlist = content.splitlines()  # 按行转换为一维表，splitlines默认参数是‘\n’
    # 逐行遍历
    # 结果按分隔符分割为行向量
    recordlist = [row.split(delimiter) for row in rowlist if row.strip()]
    return np.array(recordlist)

In [3]:
#import numpy as np
import scipy.ndimage


def ndarray_zoom_scaling(label, new_h, new_w):
    """
    Implement scaling for ndarray with scipy.ndimage.zoom
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_zoom_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 2 2 2 2 3 3 3]
     [1 1 1 2 2 2 2 3 3 3]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [4 4 4 5 5 5 5 6 6 6]
     [7 7 7 8 8 8 8 9 9 9]
     [7 7 7 8 8 8 8 9 9 9]]
    """
    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]
    if len(label.shape) == 2:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w], order=0)
    else:
        label_new = scipy.ndimage.zoom(label, zoom=[scale_h, scale_w, 1], order=0)
    return label_new


def ndarray_nearest_neighbour_scaling(label, new_h, new_w):
    """
    Implement nearest neighbour scaling for ndarray
    :param label: [H, W] or [H, W, C]
    :return: label_new: [new_h, new_w] or [new_h, new_w, C]
    Examples
    --------
    ori_arr = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]], dtype=np.int32)
    new_arr = ndarray_nearest_neighbour_scaling(ori_arr, new_h=8, new_w=10)
    >> print(new_arr)
    [[1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [1 1 1 1 2 2 2 3 3 3]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [4 4 4 4 5 5 5 6 6 6]
     [7 7 7 7 8 8 8 9 9 9]
     [7 7 7 7 8 8 8 9 9 9]]
    """
#    if len(label.shape) == 2:
#        label_new = np.zeros([new_h, new_w], dtype=label.dtype)
#    else:
#        label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)
    label_new = np.zeros([new_h, new_w, label.shape[2]], dtype=label.dtype)

    scale_h = new_h / label.shape[0]
    scale_w = new_w / label.shape[1]

    y_pos = np.arange(new_h)
    x_pos = np.arange(new_w)
    y_pos = np.floor(y_pos / scale_h).astype(np.int32)
    x_pos = np.floor(x_pos / scale_w).astype(np.int32)

    y_pos = y_pos.reshape(y_pos.shape[0], 1)
    y_pos = np.tile(y_pos, (1, new_w))
    x_pos = np.tile(x_pos, (new_h, 1))
    assert y_pos.shape == x_pos.shape

    label_new[:, :] = label[y_pos[:, :], x_pos[:, :]]
    return label_new


In [16]:
def DataSet(iFold,rand_tag):
    
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_samples_test_F%d") % (iFold, iFold)
    #
    data_train = file2array(train_path, " ")
    nrow_train = data_train.shape[0]
    ncol_train = data_train.shape[1]
    #
    data_test  = file2array(test_path, " ")
    nrow_test  = data_test.shape[0]
    ncol_test  = data_test.shape[1]
    #
    if ncol_train != ncol_test:
        print('Numbers of features are not consistant.')
    if nrow_train + nrow_test != 1567:
        print('Numbers of samples are not consistant.')
    #
    train_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_train_F%d") % (iFold, iFold)    
    test_path  =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/F%d/SECOM_labels_test_F%d") % (iFold, iFold)
    #
    lab_train = file2array(train_path, " ")
    #
    lab_test  = file2array(test_path, " ")
    #
    num_feature = ncol_train
    side_len  = math.ceil(math.sqrt(num_feature))
    num_class = 2
    #
    mat_ind_path =("../../Research/Projects/DPNeuEvo/Data_all/Data_CNN_Indus/Mat_ind_F%d.txt") % (iFold)
    #
    mat_ind = file2array(mat_ind_path, "\t")
    
    if (int)(rand_tag):
        tmp_vec = np.zeros(num_feature)
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            tmp_vec[index] = int(mat_ind[r,c])
        index = [i for i in range(num_feature)]
        random.shuffle(index)
        tmp_vec = tmp_vec[index]
        for index in range(num_feature):
            r = (int)(index/side_len)
            c = (int)(index%side_len)
            mat_ind[r,c] = int(tmp_vec[index])

    tmp_cnt_per_class = np.zeros(num_class)
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        tmp_cnt_per_class[(int)(tmp)] = tmp_cnt_per_class[(int)(tmp)] + 1
        
    tmp_max = tmp_cnt_per_class[0]
    tmp_ind = 0
    for index in range(num_class):
        if tmp_cnt_per_class[index] > tmp_max:
            tmp_max = tmp_cnt_per_class[index]
            tmp_ind = index
            
    num_dup = np.zeros(num_class)
    for index in range(num_class):
        num_dup[index] = math.floor(tmp_cnt_per_class[tmp_ind]/tmp_cnt_per_class[index])
        
    nrow_train_dup = 0
    for index in range(num_class):
        nrow_train_dup = nrow_train_dup + tmp_cnt_per_class[index] * num_dup[index]
        
    nrow_train_dup = (int)(nrow_train_dup)
    
    X_train = np.empty((nrow_train_dup, side_len, side_len, 1))
    Y_train = np.empty((nrow_train_dup, num_class))

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_cnt = 0
            for i in range(side_len):
                for j in range(side_len):
                    if tmp_cnt < num_feature:
                        tmp_ind = mat_ind[i,j]
                        X_train[sam_count,i,j,0] = data_train[index,(int)(tmp_ind)]
                    else:
                        X_train[sam_count,i,j,0] = 0.5
                    tmp_cnt = tmp_cnt + 1
            sam_count = sam_count + 1

    sam_count = 0
    for index in range(nrow_train):
        tmp = lab_train[index,0]
        nDup = num_dup[(int)(tmp)]
        nDup = (int)(nDup)
        for iSam in range(nDup):
            tmp_v = np.zeros(num_class)
            tmp_v[(int)(tmp)] = 1
            Y_train[sam_count] = tmp_v
            sam_count = sam_count + 1
    
    X_test  = np.empty((nrow_test, side_len, side_len, 1))
    Y_test  = np.empty((nrow_test, num_class))

    for index in range(nrow_test):
        tmp_cnt = 0
        for i in range(side_len):
            for j in range(side_len):
                if tmp_cnt < num_feature:
                    tmp_ind = mat_ind[i,j]
                    X_test[index,i,j,0] = data_test[index,(int)(tmp_ind)]
                else:
                    X_test[index,i,j,0] = 0.5
                tmp_cnt = tmp_cnt + 1
    
    for index in range(nrow_test):
        tmp = lab_test[index,0]
        tmp_v = np.zeros(num_class)
        tmp_v[(int)(tmp)] = 1
        Y_train[index] = tmp_v
    
    index = [i for i in range(len(X_train))]
    random.shuffle(index)
    X_train = X_train[index]
    Y_train = Y_train[index]
    
    index = [i for i in range(len(X_test))]
    random.shuffle(index)
    X_test = X_test[index]    
    Y_test = Y_test[index]
    
    return X_train,Y_train,X_test,Y_test

In [5]:
#import tensorflow as tf
def LeakyRelu(x, leak=0.01, name="LeakyRelu"):
   with tf.variable_scope(name):
     f1 = 0.5 * (1 + leak)
     f2 = 0.5 * (1 - leak)
     return f1 * x + f2 * tf.abs(x)

In [6]:
def neural(channel,height,width,classes):
    input_shape = (channel,height,width)
    if K.image_data_format() == "channels_last":#确认输入维度,就是channel是在开头，还是结尾
        input_shape = (height,width,channel)
    model = Sequential()#顺序模型（keras中包括顺序模型和函数式API两种方式）
    model.add(Conv2D(6,(3,3),padding="valid",activation=LeakyRelu,input_shape=input_shape,name="conv1"))
    #model.add(Conv2D(6,(3,3),padding="valid",input_shape=input_shape,name="conv1"))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2),name="pool1"))
    model.add(Conv2D(12,(3,3),padding="valid",activation=LeakyRelu,name="conv2",))
    #model.add(Conv2D(12,(3,3),padding="valid",name="conv2",))
    #model.add(LeakyReLU(alpha=0.01))
    model.add(AveragePooling2D(pool_size=(2,2),strides=(2,2),name="pool2"))
    model.add(Flatten())
    model.add(Dense(192,activation="relu",name="fc1"))
    model.add(Dense(classes,activation="softmax",name="fc2"))
    return model


In [7]:
models = []
for fold in range(10):
    iFold = fold + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold,0)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    #print(X_train[0])
    #print(Y_train[0,1])

    # # model
    #model = ResNet50(
    #    weights=None,
    #    classes=2
    #)
    
    ##
    model = neural(1,22,22,2)

    model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # # train
    model.fit(X_train, Y_train, epochs=100, batch_size=6)

    # # evaluate
    model.evaluate(X_test, Y_test, batch_size=32)
    
    # #
    models.append(model)
    

    # # save
    save_path =("./my_resnet_model.h5_F%d") % (iFold)
    model.save(save_path)

X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.6888 - acc: 0.5422
Epoch 2/100
2632/2632 [==============================] - 2s 635us/step - loss: 0.6631 - acc: 0.5961
Epoch 3/100
2632/2632 [==============================] - 2s 697us/step - loss: 0.5880 - acc: 0.6786
Epoch 4/100
2632/2632 [==============================] - 2s 767us/step - loss: 0.5001 - acc: 0.7618
Epoch 5/100
2632/2632 [==============================] - 2s 709us/step - loss: 0.4368 - acc: 0.8002
Epoch 6/100
2632/2632 [==============================] - 2s 734us/step - loss: 0.3787 - acc: 0.8427
Epoch 7/100
2632/2632 [==============================] - 2s 760us/step - loss: 0.3269 - acc: 0.8799
Epoch 8/100
2632/2632 [==============================] - 2s 757us/step - loss: 0.2908 - acc: 0.9016
Epoch 9/100
2632/2632 [==============================] - 2s 788us/step - loss: 0.2

2632/2632 [==============================] - 2s 786us/step - loss: 0.0417 - acc: 0.9867
Epoch 82/100
2632/2632 [==============================] - 2s 781us/step - loss: 0.0204 - acc: 0.9943
Epoch 83/100
2632/2632 [==============================] - 2s 786us/step - loss: 0.0188 - acc: 0.9935
Epoch 84/100
2632/2632 [==============================] - 2s 733us/step - loss: 0.0184 - acc: 0.9939
Epoch 85/100
2632/2632 [==============================] - 2s 771us/step - loss: 0.0586 - acc: 0.9810
Epoch 86/100
2632/2632 [==============================] - 2s 779us/step - loss: 0.0269 - acc: 0.9920
Epoch 87/100
2632/2632 [==============================] - 2s 769us/step - loss: 0.0184 - acc: 0.9935
Epoch 88/100
2632/2632 [==============================] - 2s 772us/step - loss: 0.0183 - acc: 0.9939
Epoch 89/100
2632/2632 [==============================] - 2s 784us/step - loss: 0.0184 - acc: 0.9947
Epoch 90/100
2632/2632 [==============================] - 2s 770us/step - loss: 0.0191 - acc: 0.9943
Epo

D:\toolkits.win\Anaconda3-4.4.0\envs\dlwin36-CPU\lib\site-packages\keras\models.py:124: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
2632/2632 [==============================] - 2s 817us/step - loss: 0.6914 - acc: 0.5376
Epoch 2/100
2632/2632 [==============================] - 2s 771us/step - loss: 0.6790 - acc: 0.5593
Epoch 3/100
2632/2632 [==============================] - 2s 774us/step - loss: 0.6017 - acc: 0.6862
Epoch 4/100
2632/2632 [==============================] - 2s 797us/step - loss: 0.5023 - acc: 0.7500
Epoch 5/100
2632/2632 [==============================] - 2s 769us/step - loss: 0.4478 - acc: 0.7907
Epoch 6/100
2632/2632 [==============================] - 2s 763us/step - loss: 0.3993 - acc: 0.8229
Epoch 7/100
2632/2632 [==============================] - 2s 787us/step - loss: 0.3638 - acc: 0.8507
Epoch 8/100
2632/2632 [==============================] - 2s 862us/step - loss: 0.3237 - acc: 0.8697
Epoch 9/100
2632/2632 [==============================] - 2s 744us/step - loss: 0

2632/2632 [==============================] - 3s 1ms/step - loss: 0.0217 - acc: 0.9932
Epoch 63/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0196 - acc: 0.9943
Epoch 64/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0366 - acc: 0.9890
Epoch 65/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0428 - acc: 0.9844
Epoch 66/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0225 - acc: 0.9932
Epoch 67/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0184 - acc: 0.9943
Epoch 68/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0232 - acc: 0.9920
Epoch 69/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0199 - acc: 0.9939
Epoch 70/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0309 - acc: 0.9901
Epoch 71/100
2632/2632 [==============================] - 3s 1ms/step - loss: 0.0319 - acc: 0.9886
Epoch 72/100
2632/2632 

2619/2619 [==============================] - 2s 828us/step - loss: 0.1043 - acc: 0.9675
Epoch 21/100
2619/2619 [==============================] - 2s 809us/step - loss: 0.0756 - acc: 0.9794
Epoch 22/100
2619/2619 [==============================] - 2s 802us/step - loss: 0.0729 - acc: 0.9790
Epoch 23/100
2619/2619 [==============================] - 2s 777us/step - loss: 0.0638 - acc: 0.9805
Epoch 24/100
2619/2619 [==============================] - 2s 815us/step - loss: 0.0733 - acc: 0.9779
Epoch 25/100
2619/2619 [==============================] - 2s 800us/step - loss: 0.0569 - acc: 0.9832 1s
Epoch 26/100
2619/2619 [==============================] - 2s 771us/step - loss: 0.0520 - acc: 0.9847
Epoch 27/100
2619/2619 [==============================] - 2s 870us/step - loss: 0.0621 - acc: 0.9809
Epoch 28/100
2619/2619 [==============================] - 2s 837us/step - loss: 0.0508 - acc: 0.9840
Epoch 29/100
2619/2619 [==============================] - 2s 814us/step - loss: 0.0398 - acc: 0.9878


2619/2619 [==============================] - 2s 924us/step - loss: 0.0393 - acc: 0.9855
Epoch 81/100
2619/2619 [==============================] - 3s 976us/step - loss: 0.0193 - acc: 0.9935
Epoch 82/100
2619/2619 [==============================] - 2s 925us/step - loss: 0.0155 - acc: 0.9950
Epoch 83/100
2619/2619 [==============================] - 3s 1ms/step - loss: 0.0147 - acc: 0.9954
Epoch 84/100
2619/2619 [==============================] - 2s 941us/step - loss: 0.0195 - acc: 0.9935
Epoch 85/100
2619/2619 [==============================] - 3s 981us/step - loss: 0.0274 - acc: 0.9893
Epoch 86/100
2619/2619 [==============================] - 3s 993us/step - loss: 0.0484 - acc: 0.9836
Epoch 87/100
2619/2619 [==============================] - 3s 970us/step - loss: 0.0143 - acc: 0.9954
Epoch 88/100
2619/2619 [==============================] - 2s 932us/step - loss: 0.0145 - acc: 0.9958
Epoch 89/100
2619/2619 [==============================] - 3s 974us/step - loss: 0.0138 - acc: 0.9947
Epoch

2633/2633 [==============================] - 3s 962us/step - loss: 0.0464 - acc: 0.9871
Epoch 39/100
2633/2633 [==============================] - 2s 900us/step - loss: 0.0447 - acc: 0.9871
Epoch 40/100
2633/2633 [==============================] - 2s 832us/step - loss: 0.0471 - acc: 0.9852
Epoch 41/100
2633/2633 [==============================] - 2s 905us/step - loss: 0.0449 - acc: 0.9867
Epoch 42/100
2633/2633 [==============================] - 2s 827us/step - loss: 0.0510 - acc: 0.9840
Epoch 43/100
2633/2633 [==============================] - 2s 834us/step - loss: 0.0410 - acc: 0.9894
Epoch 44/100
2633/2633 [==============================] - 2s 882us/step - loss: 0.0466 - acc: 0.9848
Epoch 45/100
2633/2633 [==============================] - 2s 840us/step - loss: 0.0478 - acc: 0.9863
Epoch 46/100
2633/2633 [==============================] - 2s 838us/step - loss: 0.0407 - acc: 0.9882
Epoch 47/100
2633/2633 [==============================] - 2s 831us/step - loss: 0.0398 - acc: 0.9882
Epo

2633/2633 [==============================] - 3s 1ms/step - loss: 0.0196 - acc: 0.9932
Epoch 99/100
2633/2633 [==============================] - 3s 1ms/step - loss: 0.0511 - acc: 0.9814
Epoch 100/100
156/156 [==============================] - 0s 1ms/step
X_train shape :  (2633, 22, 22, 1)
Y_train shape :  (2633, 2)
X_test shape :  (156, 22, 22, 1)
Y_test shape :  (156, 2)
Epoch 1/100
2633/2633 [==============================] - 3s 1ms/step - loss: 0.6880 - acc: 0.5450
Epoch 2/100
2633/2633 [==============================] - 2s 859us/step - loss: 0.6715 - acc: 0.5883
Epoch 3/100
2633/2633 [==============================] - 2s 871us/step - loss: 0.6209 - acc: 0.6540
Epoch 4/100
2633/2633 [==============================] - 2s 946us/step - loss: 0.5597 - acc: 0.7148
Epoch 5/100
2633/2633 [==============================] - 2s 917us/step - loss: 0.4926 - acc: 0.7660
Epoch 6/100
2633/2633 [==============================] - 2s 826us/step - loss: 0.4522 - acc: 0.7900
Epoch 7/100
2633/2633 [=====

In [8]:
X_train,Y_train,X_test,Y_test = DataSet(1,0)
true_labels = np.zeros(len(Y_test))
print(len(Y_test))

157


In [9]:
all_precisions=[]
all_recalls=[]
all_Fvalues=[]
all_TP=[]
all_FP=[]
all_TN=[]
all_FN=[]
nRun = 10
num_class = 2
for iRun in range(nRun):
    iFold = iRun + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold,0)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    #print(X_train[0])
    #print(Y_train[0,1])
    
    test_rows = len(Y_test)
    
    tmp_precision = np.zeros(num_class)
    tmp_recall    = np.zeros(num_class)
    tmp_Fvalue    = np.zeros(num_class)
    tmp_beta = 1
    model = models[iRun]
    yhat_label = model.predict_classes(X_test, batch_size=32, verbose=1)
    true_labels = np.zeros(len(Y_test))
    for index in range(len(Y_test)):
        tmp_vec = Y_test[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind
    
    N_TP = np.zeros(num_class)
    N_FP = np.zeros(num_class)
    N_TN = np.zeros(num_class)
    N_FN = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    N_TP[j]=N_TP[j]+1
                else:
                    N_FP[j]=N_FP[j]+1
            else:
                if j == true_label:
                    N_FN[j]=N_FN[j]+1
                else:
                    N_TN[j]=N_TN[j]+1
            
    for i in range(num_class):
        if N_TP[i] > 0:
            tmp_precision[i] = N_TP[i] / (N_TP[i] + N_FP[i])
            tmp_recall[i]    = N_TP[i] / (N_TP[i] + N_FN[i])
            tmp_Fvalue[i]=(1 + tmp_beta*tmp_beta)*tmp_recall[i]*tmp_precision[i]/\
                          (tmp_beta*tmp_beta*(tmp_recall[i]+tmp_precision[i]))
        else:
            tmp_precision[i] = 0
            tmp_recall[i] = 0
            tmp_Fvalue[i] = 0
    
    all_precisions.append(tmp_precision)
    all_recalls.append(tmp_recall)
    all_Fvalues.append(tmp_Fvalue)
    all_TP.append(N_TP)
    all_FP.append(N_FP)
    all_TN.append(N_TN)
    all_FN.append(N_FN)


X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 889us/step
X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 813us/step
X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 839us/step
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 884us/step
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 819us/step
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157

In [10]:
for iRun in range(nRun):
    print(all_precisions[iRun])
    print(all_recalls[iRun])
    print(all_Fvalues[iRun])


[0.40131579 0.2       ]
[0.93846154 0.01086957]
[0.56221198 0.02061856]
[0.42567568 0.77777778]
[0.96923077 0.07608696]
[0.5915493  0.13861386]
[0.13513514 1.        ]
[1.         0.06569343]
[0.23809524 0.12328767]
[0.13888889 1.        ]
[1.         0.09489051]
[0.24390244 0.17333333]
[0.47435897 0.        ]
[0.98666667 0.        ]
[0.64069264 0.        ]
[0.5034965  0.42857143]
[0.9        0.07792208]
[0.64573991 0.13186813]
[0.47552448 0.5       ]
[0.90666667 0.08536585]
[0.62385321 0.14583333]
[0.17105263 0.75      ]
[0.96296296 0.02325581]
[0.29050279 0.04511278]
[0.20408163 0.66666667]
[0.90909091 0.04878049]
[0.33333333 0.09090909]
[0.36486486 0.375     ]
[0.91525424 0.03092784]
[0.52173913 0.05714286]


In [11]:
for iRun in range(nRun):
    print(all_TP[iRun])
    print(all_FP[iRun])
    print(all_TN[iRun])
    print(all_FN[iRun])


[61.  1.]
[91.  4.]
[ 1. 61.]
[ 4. 91.]
[63.  7.]
[85.  2.]
[ 7. 63.]
[ 2. 85.]
[20.  9.]
[128.   0.]
[ 9. 20.]
[  0. 128.]
[20. 13.]
[124.   0.]
[13. 20.]
[  0. 124.]
[74.  0.]
[82.  1.]
[ 0. 74.]
[ 1. 82.]
[72.  6.]
[71.  8.]
[ 6. 72.]
[ 8. 71.]
[68.  7.]
[75.  7.]
[ 7. 68.]
[ 7. 75.]
[26.  3.]
[126.   1.]
[ 3. 26.]
[  1. 126.]
[30.  6.]
[117.   3.]
[ 6. 30.]
[  3. 117.]
[54.  3.]
[94.  5.]
[ 3. 54.]
[ 5. 94.]


In [12]:
tmp=np.zeros([3,num_class])
print(tmp)
print(tmp[0,0])

all_indices=[]
all_indices.append(all_precisions)
all_indices.append(all_recalls)
all_indices.append(all_Fvalues)

print(all_precisions[0][0])
print(all_indices[0][0][0])

[[0. 0.]
 [0. 0.]
 [0. 0.]]
0.0
0.40131578947368424
0.40131578947368424


In [13]:
mean_indices=np.zeros([3,num_class])
min_indices=np.ones([3,num_class])
max_indices=np.zeros([3,num_class])
std_indices=np.zeros([3,num_class])

for iInd in range(3):
    for iRun in range(nRun):
        for i in range(num_class):
            mean_indices[iInd,i] = mean_indices[iInd,i] + all_indices[iInd][iRun][i]
            if min_indices[iInd,i] > all_indices[iInd][iRun][i]:
                min_indices[iInd,i] = all_indices[iInd][iRun][i]
            if max_indices[iInd,i] < all_indices[iInd][iRun][i]:
                max_indices[iInd,i] = all_indices[iInd][iRun][i]

for iInd in range(3):
    for i in range(num_class):
        mean_indices[iInd,i] = mean_indices[iInd,i] / nRun
                
for iInd in range(3):
    for iRun in range(nRun):
        for i in range(num_class):
            std_indices[iInd,i] = (mean_indices[iInd,i] - all_indices[iInd][iRun][i])*(mean_indices[iInd,i] - all_indices[iInd][iRun][i])

for iInd in range(3):
    for i in range(num_class):
        std_indices[iInd,i] = std_indices[iInd,i] / nRun
        std_indices[iInd,i] = std_indices[iInd,i] ** 0.5


In [14]:
print(mean_indices)
print(min_indices)
print(max_indices)
print(std_indices)


[[0.32943946 0.56980159]
 [0.94883338 0.05137925]
 [0.469162   0.09267196]]
[[0.13513514 0.        ]
 [0.9        0.        ]
 [0.23809524 0.        ]]
[[0.5034965  1.        ]
 [1.         0.09489051]
 [0.64573991 0.17333333]]
[[0.0112025  0.06160167]
 [0.01061866 0.00646731]
 [0.01662635 0.01123529]]


In [17]:
models_c = []
for fold in range(10):
    iFold = fold + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold,1)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    #print(X_train[0])
    #print(Y_train[0,1])

    # # model
    #model = ResNet50(
    #    weights=None,
    #    classes=2
    #)
    
    ##
    model = neural(1,22,22,2)

    model.compile(optimizer=tf.train.AdamOptimizer(0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # # train
    model.fit(X_train, Y_train, epochs=100, batch_size=6)

    # # evaluate
    model.evaluate(X_test, Y_test, batch_size=32)
    
    # #
    models_c.append(model)
    

    # # save
    save_path =("./my_resnet_model.h5_F%d") % (iFold)
    model.save(save_path)

X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
2632/2632 [==============================] - 2s 759us/step - loss: 0.6871 - acc: 0.5426
Epoch 2/100
2632/2632 [==============================] - 2s 659us/step - loss: 0.6275 - acc: 0.6425
Epoch 3/100
2632/2632 [==============================] - 2s 644us/step - loss: 0.5470 - acc: 0.7116
Epoch 4/100
2632/2632 [==============================] - 2s 616us/step - loss: 0.4771 - acc: 0.7815
Epoch 5/100
2632/2632 [==============================] - 2s 617us/step - loss: 0.4191 - acc: 0.8024
Epoch 6/100
2632/2632 [==============================] - 2s 632us/step - loss: 0.3682 - acc: 0.8431
Epoch 7/100
2632/2632 [==============================] - 2s 637us/step - loss: 0.3210 - acc: 0.8731
Epoch 8/100
2632/2632 [==============================] - 2s 624us/step - loss: 0.2825 - acc: 0.9031
Epoch 9/100
2632/2632 [==============================] - 2s 634us/step - loss: 0

2632/2632 [==============================] - 2s 777us/step - loss: 0.0368 - acc: 0.9886
Epoch 82/100
2632/2632 [==============================] - 2s 779us/step - loss: 0.0298 - acc: 0.9913
Epoch 83/100
2632/2632 [==============================] - 2s 780us/step - loss: 0.0347 - acc: 0.9909
Epoch 84/100
2632/2632 [==============================] - 2s 762us/step - loss: 0.0320 - acc: 0.9905
Epoch 85/100
2632/2632 [==============================] - 2s 751us/step - loss: 0.0302 - acc: 0.9924
Epoch 86/100
2632/2632 [==============================] - 2s 759us/step - loss: 0.0273 - acc: 0.9924
Epoch 87/100
2632/2632 [==============================] - 2s 819us/step - loss: 0.0379 - acc: 0.9894
Epoch 88/100
2632/2632 [==============================] - 2s 765us/step - loss: 0.0344 - acc: 0.9928
Epoch 89/100
2632/2632 [==============================] - 2s 789us/step - loss: 0.0351 - acc: 0.9909
Epoch 90/100
2632/2632 [==============================] - 2s 783us/step - loss: 0.0333 - acc: 0.9913
Epo

D:\toolkits.win\Anaconda3-4.4.0\envs\dlwin36-CPU\lib\site-packages\keras\models.py:124: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
2632/2632 [==============================] - 2s 920us/step - loss: 0.6878 - acc: 0.5441
Epoch 2/100
2632/2632 [==============================] - 2s 783us/step - loss: 0.6613 - acc: 0.5912
Epoch 3/100
2632/2632 [==============================] - 2s 773us/step - loss: 0.5644 - acc: 0.7025
Epoch 4/100
2632/2632 [==============================] - 2s 792us/step - loss: 0.4765 - acc: 0.7785
Epoch 5/100
2632/2632 [==============================] - 2s 819us/step - loss: 0.4036 - acc: 0.8248
Epoch 6/100
2632/2632 [==============================] - 2s 820us/step - loss: 0.3506 - acc: 0.8568
Epoch 7/100
2632/2632 [==============================] - 2s 922us/step - loss: 0.3051 - acc: 0.8872
Epoch 8/100
2632/2632 [==============================] - 2s 869us/step - loss: 0.2536 - acc: 0.9138
Epoch 9/100
2632/2632 [==============================] - 2s 820us/step - loss: 0

2632/2632 [==============================] - 2s 803us/step - loss: 0.0224 - acc: 0.9932
Epoch 82/100
2632/2632 [==============================] - 2s 794us/step - loss: 0.0202 - acc: 0.9916
Epoch 83/100
2632/2632 [==============================] - 2s 782us/step - loss: 0.0182 - acc: 0.9951
Epoch 84/100
2632/2632 [==============================] - 2s 808us/step - loss: 0.0408 - acc: 0.9875
Epoch 85/100
2632/2632 [==============================] - 2s 827us/step - loss: 0.0205 - acc: 0.9935
Epoch 86/100
2632/2632 [==============================] - 2s 803us/step - loss: 0.0219 - acc: 0.9928
Epoch 87/100
2632/2632 [==============================] - 2s 795us/step - loss: 0.0199 - acc: 0.9943
Epoch 88/100
2632/2632 [==============================] - 2s 797us/step - loss: 0.0275 - acc: 0.9905
Epoch 89/100
2632/2632 [==============================] - 2s 817us/step - loss: 0.0199 - acc: 0.9928
Epoch 90/100
2632/2632 [==============================] - 2s 784us/step - loss: 0.0281 - acc: 0.9909
Epo

2619/2619 [==============================] - 2s 817us/step - loss: 0.0395 - acc: 0.9878
Epoch 40/100
2619/2619 [==============================] - 2s 835us/step - loss: 0.0404 - acc: 0.9874
Epoch 41/100
2619/2619 [==============================] - 2s 839us/step - loss: 0.0458 - acc: 0.9866
Epoch 42/100
2619/2619 [==============================] - 2s 828us/step - loss: 0.0381 - acc: 0.9889
Epoch 43/100
2619/2619 [==============================] - 2s 847us/step - loss: 0.0389 - acc: 0.9893
Epoch 44/100
2619/2619 [==============================] - 2s 835us/step - loss: 0.0358 - acc: 0.9893
Epoch 45/100
2619/2619 [==============================] - 2s 837us/step - loss: 0.0323 - acc: 0.9920
Epoch 46/100
2619/2619 [==============================] - 2s 853us/step - loss: 0.0355 - acc: 0.9908
Epoch 47/100
2619/2619 [==============================] - 3s 994us/step - loss: 0.0428 - acc: 0.9855
Epoch 48/100
2619/2619 [==============================] - 2s 826us/step - loss: 0.0311 - acc: 0.9889
Epo

2619/2619 [==============================] - 2s 755us/step - loss: 0.0211 - acc: 0.9939
Epoch 100/100
157/157 [==============================] - 0s 1ms/step
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
Epoch 1/100
2619/2619 [==============================] - 2s 925us/step - loss: 0.6887 - acc: 0.5368
Epoch 2/100
2619/2619 [==============================] - 2s 762us/step - loss: 0.6723 - acc: 0.5872
Epoch 3/100
2619/2619 [==============================] - 2s 771us/step - loss: 0.5823 - acc: 0.6930
Epoch 4/100
2619/2619 [==============================] - 2s 765us/step - loss: 0.4539 - acc: 0.7881
Epoch 5/100
2619/2619 [==============================] - 2s 784us/step - loss: 0.3799 - acc: 0.8358
Epoch 6/100
2619/2619 [==============================] - 2s 766us/step - loss: 0.3172 - acc: 0.8816
Epoch 7/100
2619/2619 [==============================] - 2s 761us/step - loss: 0.2685 - acc: 0.9122
Epoch 8/100
2619/2619 [

2619/2619 [==============================] - 2s 822us/step - loss: 0.0311 - acc: 0.9920
Epoch 58/100
2619/2619 [==============================] - 2s 786us/step - loss: 0.0360 - acc: 0.9901
Epoch 59/100
2619/2619 [==============================] - 2s 786us/step - loss: 0.0316 - acc: 0.9916
Epoch 60/100
2619/2619 [==============================] - 2s 794us/step - loss: 0.0318 - acc: 0.9912
Epoch 61/100
2619/2619 [==============================] - 2s 815us/step - loss: 0.0335 - acc: 0.9905
Epoch 62/100
2619/2619 [==============================] - 2s 822us/step - loss: 0.0323 - acc: 0.9905
Epoch 63/100
2619/2619 [==============================] - 2s 844us/step - loss: 0.0363 - acc: 0.9889
Epoch 64/100
2619/2619 [==============================] - 2s 818us/step - loss: 0.0285 - acc: 0.9939
Epoch 65/100
2619/2619 [==============================] - 2s 842us/step - loss: 0.0382 - acc: 0.9893
Epoch 66/100
2619/2619 [==============================] - 2s 872us/step - loss: 0.0279 - acc: 0.9927
Epo

2633/2633 [==============================] - 2s 799us/step - loss: 0.1286 - acc: 0.9624
Epoch 16/100
2633/2633 [==============================] - 2s 775us/step - loss: 0.1175 - acc: 0.9639
Epoch 17/100
2633/2633 [==============================] - 2s 797us/step - loss: 0.0995 - acc: 0.9719
Epoch 18/100
2633/2633 [==============================] - 2s 776us/step - loss: 0.0975 - acc: 0.9704
Epoch 19/100
2633/2633 [==============================] - 2s 780us/step - loss: 0.0835 - acc: 0.9753
Epoch 20/100
2633/2633 [==============================] - 2s 790us/step - loss: 0.0893 - acc: 0.9742
Epoch 21/100
2633/2633 [==============================] - 2s 762us/step - loss: 0.0742 - acc: 0.9761
Epoch 22/100
2633/2633 [==============================] - 2s 773us/step - loss: 0.0602 - acc: 0.9840
Epoch 23/100
2633/2633 [==============================] - 2s 770us/step - loss: 0.0619 - acc: 0.9795
Epoch 24/100
2633/2633 [==============================] - 2s 774us/step - loss: 0.0597 - acc: 0.9844
Epo

2633/2633 [==============================] - 2s 797us/step - loss: 0.0168 - acc: 0.9939
Epoch 75/100
2633/2633 [==============================] - 2s 793us/step - loss: 0.0194 - acc: 0.9951
Epoch 76/100
2633/2633 [==============================] - 2s 806us/step - loss: 0.0254 - acc: 0.9913
Epoch 77/100
2633/2633 [==============================] - 2s 797us/step - loss: 0.0198 - acc: 0.9932
Epoch 78/100
2633/2633 [==============================] - 2s 818us/step - loss: 0.0179 - acc: 0.9939
Epoch 79/100
2633/2633 [==============================] - 2s 830us/step - loss: 0.0198 - acc: 0.9916
Epoch 80/100
2633/2633 [==============================] - 2s 802us/step - loss: 0.0391 - acc: 0.9863
Epoch 81/100
2633/2633 [==============================] - 2s 842us/step - loss: 0.0375 - acc: 0.9852
Epoch 82/100
2633/2633 [==============================] - 2s 819us/step - loss: 0.0183 - acc: 0.9943
Epoch 83/100
2633/2633 [==============================] - 2s 836us/step - loss: 0.0165 - acc: 0.9932
Epo

In [18]:
all_precisions=[]
all_recalls=[]
all_Fvalues=[]
all_TP=[]
all_FP=[]
all_TN=[]
all_FN=[]
nRun = 10
num_class = 2
for iRun in range(nRun):
    iFold = iRun + 1
    X_train,Y_train,X_test,Y_test = DataSet(iFold,1)
    print('X_train shape : ',X_train.shape)
    print('Y_train shape : ',Y_train.shape)
    print('X_test shape : ',X_test.shape)
    print('Y_test shape : ',Y_test.shape)
    #print(X_train[0])
    #print(Y_train[0,1])
    
    test_rows = len(Y_test)
    
    tmp_precision = np.zeros(num_class)
    tmp_recall    = np.zeros(num_class)
    tmp_Fvalue    = np.zeros(num_class)
    tmp_beta = 1
    model = models_c[iRun]
    yhat_label = model.predict_classes(X_test, batch_size=32, verbose=1)
    true_labels = np.zeros(len(Y_test))
    for index in range(len(Y_test)):
        tmp_vec = Y_test[index]
        tmp_max = tmp_vec[0]
        tmp_ind = 0
        for i in range(num_class):
            if tmp_max < tmp_vec[i]:
                tmp_max = tmp_vec[i]
                tmp_ind = i
        true_labels[index] = tmp_ind
    
    N_TP = np.zeros(num_class)
    N_FP = np.zeros(num_class)
    N_TN = np.zeros(num_class)
    N_FN = np.zeros(num_class)
    for i in range(test_rows):
        cur_label = yhat_label[i]
        true_label = true_labels[i]
        for j in range(num_class):
            if j == cur_label:
                if j == true_label:
                    N_TP[j]=N_TP[j]+1
                else:
                    N_FP[j]=N_FP[j]+1
            else:
                if j == true_label:
                    N_FN[j]=N_FN[j]+1
                else:
                    N_TN[j]=N_TN[j]+1
            
    for i in range(num_class):
        if N_TP[i] > 0:
            tmp_precision[i] = N_TP[i] / (N_TP[i] + N_FP[i])
            tmp_recall[i]    = N_TP[i] / (N_TP[i] + N_FN[i])
            tmp_Fvalue[i]=(1 + tmp_beta*tmp_beta)*tmp_recall[i]*tmp_precision[i]/\
                          (tmp_beta*tmp_beta*(tmp_recall[i]+tmp_precision[i]))
        else:
            tmp_precision[i] = 0
            tmp_recall[i] = 0
            tmp_Fvalue[i] = 0
    
    all_precisions.append(tmp_precision)
    all_recalls.append(tmp_recall)
    all_Fvalues.append(tmp_Fvalue)
    all_TP.append(N_TP)
    all_FP.append(N_FP)
    all_TN.append(N_TN)
    all_FN.append(N_FN)


X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 3ms/step
X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 2ms/step
X_train shape :  (2632, 22, 22, 1)
Y_train shape :  (2632, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 1ms/step
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 1ms/step
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/157 [==============================] - 0s 1ms/step
X_train shape :  (2619, 22, 22, 1)
Y_train shape :  (2619, 2)
X_test shape :  (157, 22, 22, 1)
Y_test shape :  (157, 2)
157/1

In [ ]:
for iRun in range(nRun):
    print(all_precisions[iRun])
    print(all_recalls[iRun])
    print(all_Fvalues[iRun])


In [ ]:
for iRun in range(nRun):
    print(all_TP[iRun])
    print(all_FP[iRun])
    print(all_TN[iRun])
    print(all_FN[iRun])


In [ ]:
# # restore
#model = tf.keras.models.load_model('my_resnet_model.h5')

# # test
#img_path = "../my_nn/dataset/test/medicine/IMG_20190717_135408_BURST91.jpg"
#img_path = "../my_nn/dataset/test/glue/IMG_20190717_135425_BURST91.jpg"

#img = image.load_img(img_path, target_size=(224, 224))

#plt.imshow(img)
#img = image.img_to_array(img)/ 255.0
#img = np.expand_dims(img, axis=0)  # 为batch添加第四维

#print(model.predict(img))
#np.argmax(model.predict(img))